<a href="https://colab.research.google.com/github/devyulbae/AIClass/blob/main/2)_LLMs_novel_chain_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 패키지 설치

In [2]:
!pip install openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the

In [3]:
from pprint import pprint
from typing import Dict, List

from langchain.chains import LLMChain, SequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel


### OpenAI API key

In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


### Prompt chain 준비
* 서비스할 내용의 프롬프트 체인을 준비합니다.
* 각 프롬프트 체인을 미리 준비해 놓고, 템플릿으로 사용합니다.

In [5]:
P_PATH = "/content/drive/MyDrive/datas/multi_prompt"
IDEA_P = os.path.join(P_PATH, "extract_idea.txt")
OUTLINE_P = os.path.join(P_PATH, "write_outline.txt")
PLOT_P = os.path.join(P_PATH, "write_plot.txt")
CHAPTER_P = os.path.join(P_PATH, "write_chapter.txt")

### Prompt chain 구현
* `SequentialChain`을 이용해서 여러개의 chain을 연속적으로 구현할 수 있습니다.

In [27]:
class UserRequest(BaseModel):
    genre: str
    characters: List[Dict[str, str]]
    text: str


def read_prompt_template(file_path: str) -> str:
    with open(file_path, "r") as f:
        prompt_template = f.read()

    return prompt_template


def create_chain(llm, template_path, output_key):
    return LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template(
            template=read_prompt_template(template_path),
        ),
        output_key=output_key,
        verbose=True,
    )


def generate_novel(req: UserRequest) -> Dict[str, str]:
    writer_llm = ChatOpenAI(temperature=0.3, max_tokens=500, model="gpt-3.5-turbo")

    # 아이디어 뽑기 체인 생성
    novel_idea_chain = create_chain(writer_llm, IDEA_P, "novel_idea")

    # 아웃라인 작성 체인 생성
    novel_outline_chain = create_chain(
        writer_llm, OUTLINE_P, "novel_outline"
    )

    # 플롯 작성 체인 생성
    novel_plot_chain = create_chain(writer_llm, PLOT_P, "novel_plot")

    # 챕터 작성 체인 생성
    novel_chapter_chain = create_chain(writer_llm, CHAPTER_P, "output")

    preprocess_chain = SequentialChain(
        chains=[
            novel_idea_chain,
            novel_outline_chain,
            novel_plot_chain
        ],
        input_variables=["genre", "characters", "text"],
        output_variables=["novel_idea", "novel_outline", "novel_plot"],
        verbose=True,
    )

    context = req.dict()
    context = preprocess_chain(context)

    context["novel_chapter"] = []
    for chapter_number in range(1, 3):
        context["chapter_number"] = chapter_number
        context = novel_chapter_chain(context)
        context["novel_chapter"].append(context["output"])

    contents = "\n\n".join(context["novel_chapter"])
    return {"results": contents}

### User prompt 작성
* User가 직접 작성하는 프롬프트를 작성합니다.

In [28]:
user_data = {
    "genre": "판타지",
    "characters": [
        {
            "name": "김철수",
            "role": "주인공"
        },
        {
            "name": "이영희",
            "role": "조연"
        }
    ],
    "text": "날씨가 추워지고 있습니다."
}


* User Prompt를 입력합니다.

In [29]:
request_instance = UserRequest(**user_data)

### Text Generation

In [30]:
generate_novel(request_instance)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: [등장 인물]
[{'name': '김철수', 'role': '주인공'}, {'name': '이영희', 'role': '조연'}]

[참고 텍스트]
날씨가 추워지고 있습니다.

[등장 인물] 과 [참고 텍스트] 를 소재로 새롭고 흥미진진한 판타지 소설 아이디어를 한 문단으로 작성해줘

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: [등장 인물]
[{'name': '김철수', 'role': '주인공'}, {'name': '이영희', 'role': '조연'}]

[참고 텍스트]
날씨가 추워지고 있습니다.

[아이디어]
한 평범한 겨울 날씨에, 주인공인 김철수는 우연히 마법의 물약을 발견한다. 그 물약을 마시게 되면 날씨를 조종할 수 있는 능력을 얻게 되는데, 이를 이용해 이영희와 함께 얼음 왕국을 찾아가는 모험을 떠나게 된다. 그들은 얼음 성을 지나며 얼음 마법사와의 대결, 눈사람 군대와의 전투 등 다양한 장애물을 극복하며 진정한 용기와 우정을 발견하게 된다. 그리고 마침내 얼음 왕국의 왕과의 만남을 통해, 김철수와 이영희는 자신들의 힘을 믿고 꿈을 이루는 법을 배우게 되는데... 이들의 모험과 성장을 그린 판타지 소설 "얼음 왕국의 비밀"은 독자들에게 흥미진진한 이야기와 함께 따뜻한 겨울의 감동을 선사할 것이다.

[context]
아웃라인 단계에서는 주요 이벤트와 결말을 고려하세요. 
여기서 중요한 것은, 이 단계에서 구체적인 디테일에 매몰되기보다는 스토리의 큰 그림에 집중하는 것입니다.

[등장 인물] 과 [아이디어] 를 소재로 새롭고 흥미진진한 판타지 소설의 아웃라인을 작성해줘


> Finished chain.


> Entering new LLMChain ch

{'results': '얼음 왕국의 비밀\n\n김철수와 이영희는 마법의 물약을 마시고 얼음 왕국을 찾기 위한 여정을 시작했다. 얼음 성을 향해 걸어가는 도중, 그들은 얼음 마법사와의 대결에 직면하게 되었다. 얼음 마법사는 강력한 얼음 마법을 사용하여 김철수와 이영희를 공격했다. 하지만 김철수와 이영희는 서로를 믿고 힘을 합치며 얼음 마법사를 이기는 데에 성공했다.\n\n얼음 성을 지나며 이들은 눈사람 군대와의 전투에도 휘말렸다. 눈사람 군대는 무수한 눈덩이를 던지며 김철수와 이영희를 공격했다. 하지만 김철수와 이영희는 눈사람 군대를 속이고, 눈덩이를 이용하여 자신들을 보호하며 전투에서 승리했다.\n\n얼음 왕국의 왕과의 만남을 통해 김철수와 이영희는 자신들의 힘을 믿고 꿈을 이루는 법을 배웠다. 왕은 김철수와 이영희에게 얼음 왕국의 비밀을 알려주었다. 이 비밀은 얼음 왕국을 지키기 위해 김철수와 이영희에게 특별한 임무를 부여\n\n얼음 왕국의 수호\n\n김철수와 이영희는 얼음 왕국의 비밀을 지키기 위해 모험을 떠나야 했다. 그들은 얼음 왕국을 위협하는 악당들과의 싸움을 통해 자신들의 힘과 용기를 증명하며, 얼음 왕국을 구하기 위해 힘을 합친다.\n\n김철수와 이영희는 얼음 왕국의 비밀을 알게 된 후, 그들은 얼음 왕국을 지키기 위해 특별한 임무를 부여받았다. 이 임무는 얼음 왕국을 위협하는 악당들을 찾아내고, 그들을 물리치는 것이었다.\n\n그들은 얼음 왕국의 마법사와의 대결을 통해 자신들의 마법 능력을 발휘하며 악당들과의 전투에 임한다. 김철수는 불의 마법을 사용하여 적들을 불태우고, 이영희는 얼음의 마법을 사용하여 적들을 얼려버린다. 그들은 다양한 마법과 전략을 사용하여 악당들과의 전투에서 승리한다.\n\n하지만 악당들은 김철수와 이영희를 쉽게 포기하지 않는다. 그들은 얼음 왕국을 지키기 위해 최선을 다하는 김철수와 이영희에게 점점 더 강력한 공격을 가한다. 하지만 김철수와 이영희는 용기와 우정을 통'}